In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import CTCS2Layer, DataOutput, PlotHelper

In [3]:
nx = 100
ny = 200

dx = 20.0
dy = 20.0

dt = 1
g = 9.81
f = 0.0

rho1 = 1025.0
rho2 = 1030.0

r1 = 0.001
r2 = 0.002

A = 1


h1 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 1.5;
eta1 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u1 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
v1 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

h2 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 3.5;
eta2 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u2 = np.zeros((ny+2, nx+1), dtype=np.float32, order='C');
v2 = np.zeros((ny+1, nx+2), dtype=np.float32, order='C');

#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0
for j in range(-1, ny+1):
    for i in range(-1, nx+1):
        x = dx*i - x_center 
        y = dy*j - y_center
        size = 500.0*min(dx, dy)
        if (np.sqrt(x**2 + y**2) < size):
            ratio = h1[j+1, i+1] / (h1[j+1, i+1] + h2[j+1, i+1])
            eta1[j+1, i+1] =  0.3*np.exp(-(x**2/size+y**2/size))
            eta2[j+1, i+1] =  0#1.3*np.exp(-(x**2/size+y**2/size))
            #eta2[j+1, i+1] = np.exp(-(x**2/size+y**2/size))
            
            
#Initialize simulator
reload(CTCS2Layer)
sim = CTCS2Layer.CTCS2Layer(\
                h1, eta1, u1, v1, \
                h2, eta2, u2, v2, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r1, r2, A, \
                rho1, rho2)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
reload(PlotHelper)
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta1[1:-1, 1:-1],\
                                u1[1:-1, :], \
                                v1[:, 1:-1], \
                                eta2[1:-1, 1:-1], \
                                u2[1:-1, :], \
                                v2[:, 1:-1])


def animate(i):
    if (i>0):
        t = sim.step(10.0)
    else:
        t = 0.0
    eta1, u1, v1, eta2, u2, v2 = sim.download()

    plotter.plot(eta1[1:-1, 1:-1],\
                 u1[1:-1, :], \
                 v1[:, 1:-1], \
                 eta2[1:-1, 1:-1],\
                 u2[1:-1, :], \
                 v2[:, 1:-1])
    fig.suptitle("CTCS-2 layer Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / 50.0) + " % => t=" + str(t)
        fig.savefig("{:010.0f}_ctcs2l.png".format(t))
            
anim = animation.FuncAnimation(fig, animate, range(100), interval=100)
plt.close(anim._fig)
anim

Using  GeForce 840M
000 % => t=0.0
000 % => t=0.0
020 % => t=100.0
040 % => t=200.0
060 % => t=300.0
080 % => t=400.0
100 % => t=500.0
120 % => t=600.0
140 % => t=700.0
160 % => t=800.0
180 % => t=900.0
